### Descarga de Librerias

In [1]:
!git clone https://github.com/maximecb/gym-minigrid.git
%cd gym-minigrid
!python setup.py install
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0
!pip3 install box2d-py

Cloning into 'gym-minigrid'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1777 (delta 3), reused 1 (delta 0), pack-reused 1767
Receiving objects: 100% (1777/1777), 7.11 MiB | 36.79 MiB/s, done.
Resolving deltas: 100% (1218/1218), done.
/content/gym-minigrid
running install
running bdist_egg
running egg_info
creating gym_minigrid.egg-info
writing gym_minigrid.egg-info/PKG-INFO
writing dependency_links to gym_minigrid.egg-info/dependency_links.txt
writing requirements to gym_minigrid.egg-info/requires.txt
writing top-level names to gym_minigrid.egg-info/top_level.txt
writing manifest file 'gym_minigrid.egg-info/SOURCES.txt'
writing manifest file 'gym_minigrid.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/gym_minigrid
copying gym_minigrid/wrappers.py -> build/lib/gy

### Importar Librerias

In [2]:
import gym
import gym_minigrid
import numpy as np
from stable_baselines.common.vec_env import *
from stable_baselines.common import set_global_seeds
from stable_baselines import PPO2, DDPG, DQN, A2C, ACER, TRPO
from stable_baselines.common.policies import MlpPolicy, CnnPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnLstmPolicy, CnnLnLstmPolicy
# from stable_baselines.deepq.policies import MlpPolicy, CnnPolicy
from stable_baselines.common.evaluation import evaluate_policy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### Carga de Entorno Minigrid

In [3]:
# Eenvironement for learning
# Use a separate environement for evaluation
from gym_minigrid.wrappers import *

env_id = 'MiniGrid-DoorKey-5x5-v0'
# env = make_vec_env(env, n_envs=2)
# env = RGBImgObsWrapper(env) # Get pixel observations
# env = ImgObsWrapper(env) # Get rid of the 'mission' field
# env = DirectionObsWrapper(env)
# env = FlatObsWrapper(env)
# env = gym.make('MiniGrid-DoorKey-5x5-v0')
# print(env.observation_space)
# print(env.action_space) # 7
eval_env = gym.make(env_id)
# vectorized environments allow to easily multiprocess training
# we demonstrate its usefulness in the next examples


In [4]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env = FlatObsWrapper(env)
        # Important: use a different seed for each environment
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init

In [13]:
# env = DummyVecEnv([make_env(env_id, i) for i in range(7)])
env = SubprocVecEnv([make_env(env_id, i) for i in range(7)], start_method='spawn')
train_env = VecNormalize(env)
train_env.close()

ConnectionResetError: ignored

In [ ]:
%cd /content/gym-minigrid/

/content/gym-minigrid


In [ ]:
# Use a separate environement for evaluation
# eval_env = gym.make('CartPole-v1')

# Random Agent, before training
# mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

# print(f"mean_reward for random agent:{mean_reward:.2f} +/- {std_reward:.2f}")

### Definicion de Modelo del Agente

In [6]:

agent_model = PPO2(MlpPolicy, train_env, ent_coef=0.0, nminibatches=32, noptepochs=10, verbose=1, tensorboard_log="./ppo2_minigriddoorkey/")





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





### Evaluar Agente Ingenuo/Aleatorio

In [8]:
# Random Agent, before training using property
from stable_baselines.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(agent_model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

ValueError: ignored

### Entrenar Agente

In [48]:
agent_model.learn(total_timesteps=100000, log_interval=1000)
env.close()

------------------------------------
| approxkl           | 0.004593449 |
| clipfrac           | 0.059821397 |
| explained_variance | 0.132       |
| fps                | 388         |
| n_updates          | 1           |
| policy_entropy     | 1.9081484   |
| policy_loss        | -0.01233819 |
| serial_timesteps   | 128         |
| time_elapsed       | 2.22e-05    |
| total_timesteps    | 896         |
| value_loss         | 0.35355     |
------------------------------------


### Evaluar Agente Entrenado

In [55]:
# Trained Agent

mean_reward, std_reward = evaluate_policy(agent_model, train_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

AssertionError: ignored

In [4]:
!pip install tensorboard==2.0.0

  Using cached https://files.pythonhosted.org/packages/9b/a6/e8ffa4e2ddb216449d34cfcb825ebb38206bee5c4553d69e7bc8bc2c5d64/tensorboard-2.0.0-py3-none-any.whl
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.0.0 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0


In [ ]:
!tensorboard --logdir ./ppo2_minigriddoorkey/ --host localhost 

### Extras

In [ ]:
import gym_minigrid

env = gym.make('MiniGrid-SimpleCrossingEnvUmaze-v0')
for i_episode in range(3):
    obs = env.reset()
    for t_steps in range(100):
        # env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        print(obs)
        if done:
            print("Episode finished after {} timesteps".format(t_steps+1))
            break
env.close()

In [ ]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper

In [10]:
def evaluate(model, eval_env, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    # env = gnwrapper.LoopAnimation(model.get_env()) # Start Xvfb
    env = eval_env
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            # env.render()
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
        

    # env.display()
    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [11]:
# Random Agent, before training using function
mean_reward_before_train = evaluate(agent_model, eval_env, num_episodes=100)

ValueError: ignored